In [1]:
! pip install -q tensorflow numpy pandas scikit-learn mlxtend dataprep transformers
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./

     |████████████████████████████████| 112kB 2.8MB/s 
     |████████████████████████████████| 890kB 7.8MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 1.2MB 9.5MB/s 
     |████████████████████████████████| 2.2MB 20.2MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 368kB 21.5MB/s 
     |████████████████████████████████| 829kB 31.5MB/s 
     |████████████████████████████████| 512kB 50.6MB/s 
     |████████████████████████████████| 8.6MB 44.1MB/s 
     |████████████████████████████████| 1.4MB 48.3MB/s 
     |████████████████████████████████| 5.5MB 21.3MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
     |████████████████████████████████| 1.1MB 47.6MB/s 
     |████████████████████████████████| 3.0MB 50.3MB/s 
     |████████████████████████████████| 153kB 2.8MB/s 
     |████████████████████████████████| 266kB 8.8MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
   

In [2]:
def augment(training):
  training = np.repeat(training, 10, axis=0)
  temp = pickle.load(open('alexnet_embeddings.pkl', 'rb'))
  for i in range(0, len(training), 10):
    for j in range(len(X)):
        if np.all(training[i][:1000] == X[j][:1000]):
          for k in range(1, 10):
            training[i+k] = np.concatenate([temp[j*10+k][:1000], training[i+k][1000:]])
          break
  return training

In [3]:
def ds_from_df(df, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Meme')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [4]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [5]:
def get_X(temp):
  res = []
  for i in range(0, len(temp), 10):
    res.append(temp[i])
  return np.array(res)

In [6]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [7]:
def build_model(input_shape=(None, 2889)):
  model = tf.keras.Sequential([tf.keras.layers.Dense(1024, 
                                                     name='dense_1',
                                                     activation=tf.keras.activations.relu),
                               tf.keras.layers.Dropout(rate=0.3),
                               tf.keras.layers.Dense(512, 
                                                     name='dense_2',
                                                     activation=tf.keras.activations.relu),
                               tf.keras.layers.Dense(512, 
                                                     name='dense_3',
                                                     activation=tf.keras.activations.relu),
                               tf.keras.layers.Dense(128, 
                                                     name='dense_4',
                                                     activation=tf.keras.activations.relu),
                               tf.keras.layers.Dense(64, 
                                                     name='dense_5',
                                                     activation=tf.keras.activations.relu),
                               tf.keras.layers.Dense(1, name='output',
                                                     activation=tf.keras.activations.sigmoid)])
  print(model.compute_output_shape(input_shape=input_shape))
  model.build(input_shape=input_shape)
  return model

In [62]:
import pandas as pd

df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)

In [98]:
import numpy as np
import pickle

### Alternate these three, in order to dervie each feedforward network!

# X, y = np.array([embedding[1][i].split() for i in range(1600)]).astype(float), df[['Meme']].values
# X, y = get_X(pickle.load(open('alexnet_embeddings.pkl', 'rb'))), df[['Meme']].values
# X, y = pickle.load(open('densenest_embeddings.pkl', 'rb')), df[['Meme']].values

In [99]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

ssc = StandardScaler()
mms = MinMaxScaler()

In [100]:
import pickle
from datetime import date

temp = mms.fit_transform(np.array([(date(int(i[0].split('-')[0]), int(i[0].split('-')[1]), int(i[0].split('-')[2])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(1600, 1))
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = ssc.fit_transform(df[['Engagement']].values)
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = ohe(create_map())
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = pickle.load(open('cls_umberto_embeddings.pkl', 'rb'))
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

(1600, 1)
(1600, 1001)
(1600, 1)
(1600, 1002)
(1600, 71)
(1600, 1073)
(1600, 768)
(1600, 1841)


In [81]:
buffer_size = 10000
batch_size = 64
num_epochs = 100

In [101]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
# X_train = augment(X_train)
# y_train = np.repeat(y_train, 10, axis=0)
print(X.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

ds_train = ds_train.shuffle(buffer_size=buffer_size,
                            reshuffle_each_iteration=False)
ds_test = ds_test.batch(batch_size)
ds_train = ds_train.batch(batch_size)

(1600, 1841) (1280, 1841) (1280, 1) (320, 1841) (320, 1)


In [102]:
model = build_model(input_shape=(None, 1841)) ## Modify input shape!
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])
model.summary()

(None, 1)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              1886208   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
output (Dense)               (None, 1)      

In [103]:
hist = model.fit(ds_train,
                 validation_data=ds_test,
                 epochs=num_epochs,
                 batch_size=batch_size,
                 use_multiprocessing=True,
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, restore_best_weights=True)])

Epoch 1/100
20/20 [==============================] - 1s 38ms/step - loss: 0.6210 - binary_accuracy: 0.6687 - val_loss: 0.4734 - val_binary_accuracy: 0.7688
Epoch 2/100
20/20 [==============================] - 1s 34ms/step - loss: 0.5085 - binary_accuracy: 0.7469 - val_loss: 0.4534 - val_binary_accuracy: 0.7844
Epoch 3/100
20/20 [==============================] - 1s 34ms/step - loss: 0.4725 - binary_accuracy: 0.7836 - val_loss: 0.4874 - val_binary_accuracy: 0.7563
Epoch 4/100
20/20 [==============================] - 1s 34ms/step - loss: 0.4401 - binary_accuracy: 0.7969 - val_loss: 0.4410 - val_binary_accuracy: 0.7875
Epoch 5/100
20/20 [==============================] - 1s 34ms/step - loss: 0.4085 - binary_accuracy: 0.8172 - val_loss: 0.4273 - val_binary_accuracy: 0.8188
Epoch 6/100
20/20 [==============================] - 1s 34ms/step - loss: 0.3918 - binary_accuracy: 0.8281 - val_loss: 0.4097 - val_binary_accuracy: 0.8094
Epoch 7/100
20/20 [==============================] - 1s 33ms/ste

In [97]:
model.save('models/alexnet_ffwd')

INFO:tensorflow:Assets written to: models/alexnet_ffwd/assets


In [111]:
! cp -r models ./drive/My\ Drive/Colab\ Notebooks/